In [11]:
import pandas as pd
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm

from selenium.webdriver import Chrome
from time import sleep

from sklearn.ensemble import RandomForestRegressor

In [21]:
clf = pickle.load(open('model.pkl', 'rb'))

In [22]:
raceid = '202306010809'

In [ ]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')             
options.add_argument('--disable-extensions')       
options.add_argument('--proxy-server="direct://"') 
options.add_argument('--proxy-bypass-list=*')      
options.add_argument('--start-maximized')      
driver = Chrome(executable_path="", options=options)

target_url = "https://regist.netkeiba.com/account/?pid=login"
sleep(1)
driver.get(target_url)
sleep(1)

USERNAME = ""
PASSWORD = ""

username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[1]/input')
sleep(1)
username_input[0].send_keys(USERNAME)
sleep(1)

username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[2]/input')
sleep(1)
username_input[0].send_keys(PASSWORD)
sleep(1)

username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/div[1]/input')
sleep(1)
username_input[0].click()
sleep(1)

rece = driver.get('https://db.netkeiba.com/race/' + raceid)
sleep(1)

table = driver.find_elements(By.XPATH,'//*[@id="contents_liquid"]/table')
sleep(1)
html = table[0].get_attribute('outerHTML')

df = pd.read_html(html)[0]


In [24]:
resultDf = df.copy()

In [25]:
df.drop(['タイム', '着差', '着順', '馬名', 'ﾀｲﾑ指数', '通過', '上り', '調教ﾀｲﾑ', '厩舎ｺﾒﾝﾄ', '賞金(万円)','備考'], axis=1, inplace=True)
df['性別'] = df['性齢'].str[0]
df['年齢'] = df['性齢'].str[1:3]
df['年齢'] = df['年齢'].astype(int)
df.drop('性齢', axis=1,inplace=True)
df['場所'] =df['調教師'] .str[1:2]
df['調教師'] = df['調教師'].str[4:]
result = []
for i  in df['馬体重'].str[4:].str.replace(')','', regex=True).str[0]:
    if i=='+':
        result.append(1)
    elif i=='-':
        result.append(2)
    else:
        result.append(0)

df['加減'] = result
df['体重増減'] = df['馬体重'].str[4:].str.replace(')','', regex=True).str[1:]
df.loc[(df.体重増減 == ''), '体重増減'] = '0'
df['体重増減'] = df['体重増減'].astype(int)
df['馬体重'] = df['馬体重'].str[0:3]
# df.loc[(df.馬体重 !='計不'), ['馬体重']].astype(float).mean().round().astype(int)
df.loc[(df.馬体重=='計不'), '馬体重'] = '471'
df['馬体重'] = df['馬体重'].astype(int)
df.loc[(df.単勝=='---'), '単勝'] = 0
df['単勝'] = df['単勝'].astype(float).round().astype(int)

In [26]:
dummy = df[['場所','性別']]
dummy = pd.get_dummies(dummy)
newDf = pd.concat([df, dummy], axis=1)
newDf.drop(['騎手','場所','性別','調教師','馬主'],axis=1,inplace=True)

In [29]:
newDf

,枠番,馬番,斤量,単勝,人気,馬体重,年齢,加減,体重増減,場所_東,場所_西,性別_牡,場所_地,場所_外,性別_セ
0,2,2,56,6,4,478,3,2,2,1,0,1,0,0,0
1,4,4,56,7,5,484,3,1,10,0,1,1,0,0,0
2,5,5,56,5,3,542,3,1,18,1,0,1,0,0,0
3,7,8,56,63,8,488,3,2,2,0,1,1,0,0,0
4,8,9,56,5,2,498,3,2,4,1,0,1,0,0,0
5,7,7,56,2,1,460,3,1,4,1,0,1,0,0,0
6,3,3,56,26,6,438,3,1,4,1,0,1,0,0,0
7,6,6,56,53,7,464,3,1,12,0,1,1,0,0,0
8,8,10,56,188,10,478,3,1,26,1,0,1,0,0,0
9,1,1,56,176,9,454,3,2,4,1,0,1,0,0,0


In [27]:
newDf['場所_地'] = 0
newDf['場所_外'] = 0
newDf['性別_セ'] = 0
newDf['性別_牝'] = 0

In [31]:
result = pd.DataFrame(clf.predict_proba(newDf),columns=['出遅れない', '出遅れる'])

In [32]:
pd.concat([result,resultDf[['馬番','馬名','人気']]], axis=1).sort_values(by='出遅れる', ascending=False)

,出遅れない,出遅れる,馬番,馬名,人気
8,0.589770,0.410230,10,コンチトーホク,10
7,0.714125,0.285875,6,オリオンネビュラ,7
6,0.753901,0.246099,3,トーセントラム,6
3,0.755704,0.244296,8,マイネルカーライル,8
9,0.759983,0.240017,1,トーセンウォルト,9
1,0.793553,0.206447,4,ショウナンアレクサ,5
2,0.801424,0.198576,5,ネイビースター,3
5,0.814148,0.185852,7,スズハローム,1
0,0.821591,0.178409,2,パクスオトマニカ,4
4,0.832458,0.167542,9,レッドロスタム,2
